# Create source plates for media preparation using biomek

The notebook also generates files with instructions on how to prepare source plates with stock solutions, culture, antiobiotics.

Tested using **ART 3.9.4** kernel on jprime.lbl.gov

## Inputs and outputs

#### Required files to run this notebook:
   
   - `../data/stock_concentrations.csv` 
   

#### Files generated by running this notebook:


   - `24-well_stock_plate_high.csv`, `24-well_stock_plate_low.csv` - instructions on how to prepare the source plates
   
   - `24-well_stock_plate_fresh.csv` - the plate that includes components that need to be prepared fresh for every cycle (culture, FeSO4)
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [1]:
import os
import sys
sys.path.append('../')

import pandas as pd
import numpy as np


## User parameters

In [2]:
user_params = {
    'stock_conc_file': '../data/stock_concentrations_extended.csv',
    'output_path': f'../data/',  # Path for output files
}

Load the stock concentrations

In [3]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")

In [4]:
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],2000.0000,2000.000,1.0
Tricine[mM],400.0000,400.000,1.0
H3BO3[mM],0.1200,4.800,40.0
Glucose[mM],3000.0000,3000.000,1.0
K2SO4[mM],3.0000,60.000,20.0
K2HPO4[mM],79.2000,792.000,10.0
FeSO4[mM],0.3000,6.000,20.0
NH4Cl[mM],1920.0000,1920.000,1.0
MgCl2[mM],7.8000,156.000,20.0


## Create stock solutions plate dataframe


In [5]:
df_stock_plate = pd.DataFrame(
    columns=["Component", "Stock", "Concentration"])
    
i = 0
num_comp = len(df_stock)
for i in range(2*num_comp):

    if i < num_comp:
        component = df_stock.index[i]
        df_stock_plate.loc[i] = [
            component, 
            "high",
            df_stock.loc[component]["High Concentration"]
        ]
    elif i < 2*num_comp:
        component = df_stock.index[(i-num_comp)]
        if df_stock.loc[component]["Dilution Factor"] > 1.0:
            df_stock_plate.loc[i] = [
                component, 
                "low",
                df_stock.loc[component]["Low Concentration"]
            ]
            
df_stock_plate.reset_index(drop=True, inplace=True)

In [6]:
df_stock_plate

,Component,Stock,Concentration
0,MOPS[mM],high,2000.0000
1,Tricine[mM],high,400.0000
2,H3BO3[mM],high,4.8000
3,Glucose[mM],high,3000.0000
4,K2SO4[mM],high,60.0000
5,K2HPO4[mM],high,792.0000
6,FeSO4[mM],high,6.0000
7,NH4Cl[mM],high,1920.0000
8,MgCl2[mM],high,156.0000
9,NaCl[mM],high,3750.0000


Split the components into two source plates - one with high concentration levels and the other one with low levels

#### High level

In [7]:
df_stock_plate_high = df_stock_plate.copy()
df_stock_plate_high = df_stock_plate_high[df_stock_plate_high['Stock']=='high']
df_stock_plate_high.drop(columns='Stock', inplace=True)


Remove FeSO4 from this plate as we need to make a fresh solution for every run.

In [8]:
df_stock_plate_high.drop(df_stock_plate_high[df_stock_plate_high['Component']=='FeSO4[mM]'].index, inplace=True)


Create additional wells of NaCl, as it will be used in larger volumes:

In [9]:
df_stock_plate_high = pd.concat([df_stock_plate_high, df_stock_plate_high[df_stock_plate_high['Component']=='NaCl[mM]']], ignore_index=True)

In [10]:
df_stock_plate_high

,Component,Concentration
0,MOPS[mM],2000.000
1,Tricine[mM],400.000
2,H3BO3[mM],4.800
3,Glucose[mM],3000.000
4,K2SO4[mM],60.000
5,K2HPO4[mM],792.000
6,NH4Cl[mM],1920.000
7,MgCl2[mM],156.000
8,NaCl[mM],3750.000
9,(NH4)6Mo7O24[mM],0.036


Assign well names:

In [11]:
num_source_wells = len(df_stock_plate_high) 

source_well_type = '24-well'
well_rows = 'ABCD'
well_columns = '123456'
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_high.reset_index(drop=True, inplace=True)
df_stock_plate_high['Well'] = well_names
df_stock_plate_high = df_stock_plate_high.set_index(['Well'])
df_stock_plate_high

,Component,Concentration
Well,,
A1,MOPS[mM],2000.000
B1,Tricine[mM],400.000
C1,H3BO3[mM],4.800
D1,Glucose[mM],3000.000
A2,K2SO4[mM],60.000
B2,K2HPO4[mM],792.000
C2,NH4Cl[mM],1920.000
D2,MgCl2[mM],156.000
A3,NaCl[mM],3750.000


#### Low level

In [12]:
df_stock_plate_low = df_stock_plate.copy()
df_stock_plate_low = df_stock_plate_low[df_stock_plate_low['Stock']=='low']
df_stock_plate_low.drop(columns='Stock', inplace=True)
df_stock_plate_low.drop(df_stock_plate_low[df_stock_plate_low['Component']=='FeSO4[mM]'].index, inplace=True)
df_stock_plate_low

,Component,Concentration
17,H3BO3[mM],0.1200
18,K2SO4[mM],3.0000
19,K2HPO4[mM],79.2000
21,MgCl2[mM],7.8000
22,NaCl[mM],1500.0000
23,(NH4)6Mo7O24[mM],0.0009
24,CoCl2[mM],0.0090
25,CuSO4[mM],0.0030
26,MnSO4[mM],0.0240
27,ZnSO4[mM],0.0030


Assign well names:

In [13]:
num_source_wells = len(df_stock_plate_low) 

well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_low.reset_index(drop=True, inplace=True)
df_stock_plate_low['Well'] = well_names
df_stock_plate_low = df_stock_plate_low.set_index(['Well'])
df_stock_plate_low

,Component,Concentration
Well,,
A1,H3BO3[mM],0.1200
B1,K2SO4[mM],3.0000
C1,K2HPO4[mM],79.2000
D1,MgCl2[mM],7.8000
A2,NaCl[mM],1500.0000
B2,(NH4)6Mo7O24[mM],0.0009
C2,CoCl2[mM],0.0090
D2,CuSO4[mM],0.0030
A3,MnSO4[mM],0.0240


#### Fresh stocks plate

Create a plate with fresh stocks of FeSO4 and culture:

In [14]:
fe_low = df_stock_plate[(df_stock_plate['Component']=='FeSO4[mM]') & (df_stock_plate['Stock']=='low')]['Concentration'].values[0]
fe_high = df_stock_plate[(df_stock_plate['Component']=='FeSO4[mM]') & (df_stock_plate['Stock']=='high')]['Concentration'].values[0]

df_stock_plate_fresh = pd.DataFrame(data=[['A1', 'Culture', ''], ['B1', 'FeSO4[mM]', fe_low], ['C1', 'FeSO4[mM]', fe_high]],
                                    columns=['Well', 'Component', 'Concentration'], 
                                    )
df_stock_plate_fresh =df_stock_plate_fresh.set_index("Well")
df_stock_plate_fresh

,Component,Concentration
Well,,
A1,Culture,
B1,FeSO4[mM],0.3
C1,FeSO4[mM],6.0


## Save source plate instructions:

In [15]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate_high_extended.csv"
df_stock_plate_high.to_csv(stock_plate_file)

In [16]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate_low_extended.csv"
df_stock_plate_low.to_csv(stock_plate_file)

In [17]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate_fresh_extended.csv"
df_stock_plate_fresh.to_csv(stock_plate_file)